In [ ]:
from data_preprocessing.DataLoader import DataLoader
import pandas as pd
import matplotlib.pyplot as plt
import calendar

In [ ]:
d = DataLoader.load_data_from_path("data/data_new")

In [ ]:
products = d.products

In [ ]:
# Splitting category_path into smaller categories, so it can be easier to group them and analyze each category separately

categories = products.category_path.str.split(';', expand=True)
products = pd.concat([products, categories], axis=1)
products = products.drop(columns=['category_path'])
products = products.rename(columns= {0: "primary_category", 1 :"secondary_category", 2 : "tertiary_category", 3 : "quaternary_category"})
products['price'] = products['price'].abs()
print(products.quaternary_category)

In [ ]:
products.dtypes

In [ ]:
products.describe()

In [ ]:
deliveries = d.deliveries

In [ ]:
deliveries

In [ ]:
deliveries.dtypes

In [ ]:
deliveries

In [ ]:
deliveries.loc[deliveries["delivery_company"].isnull(),["delivery_company"]] = "Empty"
# deliveries["delivery_company"].isnull()="Empty"

In [ ]:
print(deliveries.to_string())

In [ ]:
#count difference between ordering and delivering
deltas =  deliveries.delivery_timestamp - deliveries.purchase_timestamp 

In [ ]:
#get rid of None values
deltas_clean = deltas.dropna()

In [ ]:
deltas_clean

In [ ]:
#take absolute values from cleand data deltas
deltas_clean_abs = deltas_clean.abs()

In [ ]:
#delivery time without absolute values
deltas_clean.astype('timedelta64[D]').plot.hist(bins = 10)

In [ ]:
#delivery time without negative values
deltas_clean[deltas_clean>= pd.Timedelta(0)].astype('timedelta64[D]').plot.hist(bins = 10)

In [ ]:
#delivery time with absolute values
deltas_clean_abs.astype('timedelta64[D]').plot.hist(bins = 10)

In [ ]:
deliveries["deltas"] = deltas.astype('timedelta64[D]')

In [ ]:
deliveries_clean = deliveries.dropna()

In [ ]:
import numpy as np

In [ ]:
#mean and standard variation of each company's delivery duration
deliveries_clean.groupby("delivery_company")["deltas"].agg([np.mean,np.std])

In [ ]:
deliveries_clean.boxplot(column="deltas",by='delivery_company',figsize=(10,10))

In [ ]:
deliveries.describe()

In [ ]:
# Client location analysis - knowing where clients live is crucial to estimate delivery time
# The smaller the city is, the delivery time gets longer
cityCount = d.users.groupby('city', as_index=False).user_id.count()
plt.figure(figsize=(10, 8))
plt.bar(cityCount.city, cityCount.user_id, color='#BE5D77')
plt.title("User count by city")
plt.xlabel("Cities")
plt.ylabel("User count")

In [ ]:
# Joining dataframes to further use in other plots
# We want to check whether there is a relationship between cities and delivery time.
deliveries_ses_user = deliveries_clean.merge(d.sessions[['user_id', 'purchase_id', 'product_id']], on='purchase_id')
deliveries_ses_user = deliveries_ses_user.merge(d.users[['user_id','city']], on='user_id')

In [ ]:
deliveries_ses_user.boxplot(column="deltas",by='city',figsize=(10,10))

In [ ]:
# Turned out boxplots weren't helpful so we check the mean for each city
deliveries_city = deliveries_ses_user.groupby('city', as_index=False).deltas.mean()
plt.figure(figsize=(10, 8))
plt.bar(deliveries_city.city, deliveries_city.deltas, color='#BE5D77')
plt.title("Delivery mean by city")
plt.xlabel("Cities")
plt.ylabel("Deliveries mean")

In [ ]:
#Joining products dataframe to check the relationship between delivery time and product category.
deliveries_ses_user_prod = deliveries_ses_user.merge(products, on='product_id')


In [ ]:
deliveries_ses_user_prod.boxplot(column="deltas",by='primary_category',figsize=(10,10))

In [ ]:
#Checking the mean for categories as well
deliveries_category = deliveries_ses_user_prod.groupby('primary_category', as_index=False).deltas.mean()
plt.figure(figsize=(10, 8))
plt.bar(deliveries_category.primary_category, deliveries_category.deltas, color='#BE5D77')
plt.title("Delivery mean by category")
plt.xlabel("Categories")
plt.ylabel("Deliveries mean")

In [ ]:
#Adding new column with day of the week and checking its relationship with delivery time
deliveries_ses_user_prod['weekday'] = deliveries_ses_user_prod['purchase_timestamp'].dt.day_name()
deliveries_ses_user_prod.boxplot(column="deltas",by='weekday',figsize=(10,10), positions=[4,0,5,6,3,1,2])


In [ ]:
# No simple way to reorder the bars in matplotlib, so we had to use this trick
# We create an artificial key, by which we will sort dataframe
work_df = deliveries_ses_user_prod
work_df = work_df.groupby('weekday').deltas.mean().reset_index()
work_df = work_df.reindex([1,5,6,4,0,2,3])
plt.figure(figsize=(10, 8))

plt.bar(work_df.weekday, work_df.deltas, color='#BE5D77')
plt.title("Delivery mean by day of the week the item was purchased")
plt.xlabel("Days of the week")
plt.ylabel("Deliveries mean")

In [ ]:
#Function for labeling rows
def labelTimeOfDay(row):
    hour = row['purchase_timestamp'].hour
    if(hour >= 6 and hour < 12):
        return "Morning"
    elif(hour >= 12 and hour < 18):
        return "Afternoon"
    elif(hour >=18 and hour < 24):
        return "Evening"
    else:
        return "Night"

In [ ]:
#Using the function above to label all the rows
deliveries_clean.loc[:,'time_of_day'] = deliveries_clean.apply(lambda row: labelTimeOfDay(row), axis=1)

In [ ]:
#Checking the relationship between the time of the day when we purchased the item and its delivery time
deliveries_clean.boxplot(column="deltas",by='time_of_day',figsize=(10,10), positions=[1,2,0,3])

In [ ]:
# Once again checking the mean for other clues
deliveries_clean = deliveries_clean.groupby('time_of_day', as_index=False).deltas.mean()
plt.figure(figsize=(10, 8))
plt.bar(deliveries_clean.time_of_day, deliveries_clean.deltas, color='#BE5D77')
plt.title("Delivery mean by time of the day the item was purchased")
plt.xlabel("Time of day")
plt.ylabel("Deliveries mean")

In [ ]:
# Investigating negative values of delta column 
deliveries_ses_user_prod_neg = deliveries_ses_user_prod[deliveries_ses_user_prod['deltas'] < 0]

In [ ]:
deliveries_ses_user_prod

In [ ]:
# Rows with negative values represent around 31% of the whole dataset 
percent = len(deliveries_ses_user_prod_neg.index)/len(deliveries_ses_user_prod.index)
print(round(percent,2))

In [ ]:
# Analyzing the problem of NaT values in timestamps columns in deliveries dataframe
deliveries = d.deliveries
deliveries_empty = deliveries[deliveries.purchase_timestamp.isnull() | deliveries.delivery_timestamp.isnull()]
percent = len(deliveries_empty)/len(deliveries)
print(round(percent,2))

In [ ]:
# There is no problem with checking when the item was purchased on the site
print(len(deliveries[deliveries.purchase_timestamp.isnull()]))

In [ ]:
delivery_empty = deliveries[deliveries.delivery_timestamp.isnull()]
delivery_grouped = deliveries_empty.groupby('delivery_company', as_index=False).purchase_id.count()

In [ ]:
plt.figure(figsize=(10, 8))
plt.bar(delivery_grouped.delivery_company.astype(str), delivery_grouped.purchase_id, color='#BE5D77')
plt.title("NaT value in delivery timestamp column")
plt.xlabel("Delivery company")
plt.ylabel("Number of empty rows")

In [ ]:
delivery_empty

In [ ]:
# Trying to understand why there are so many empty rows
deliveries = d.deliveries
deliveries_empty = deliveries[deliveries.isna().any(axis=1)]

In [ ]:
# Searching for last delivery by each company
deliveries_grouped = deliveries.groupby('delivery_company')['delivery_timestamp']
deliveries_empty = deliveries_empty.assign(max=deliveries_grouped.transform(max))
deliveries_empty.loc[:,'max_delta'] = deliveries_empty['max'] - deliveries_empty['purchase_timestamp']
deliveries_empty.dtypes
deliveries_empty["max_delta"] = deliveries_empty["max_delta"].astype('timedelta64[D]')
deliveries_empty.boxplot(column="max_delta",by='delivery_company',figsize=(10,10))

In [ ]:
#sklearn is used only to split in a good way data
from sklearn.model_selection import train_test_split
train, test = train_test_split(deliveries_ses_user_prod, test_size=0.2)

In [ ]:
train